# LLM-for-Metadata-Harvesting

This notebook demonstrates the use of Large Language Models (LLMs) for automated metadata extraction from web-based dataset portals.  
It showcases an experiment on the [Actual probability distribution for Quercus robur (2000–2020)](https://stac.ecodatacube.eu/veg_quercus.robur_anv.eml/collection.json?.language=en) dataset, using a combination of web scraping and LLM-based entity extraction to populate metadata fields according to the Croissant data standard.

Key features:
- Web scraping utilities for extracting full page text from dataset portals
- Environment configuration for flexible API and model usage
- LLM client support for OpenAI and Gemini models (with extensibility for custom clients)
- Automated extraction of core metadata fields such as description, license, creator, keywords, and more

The results of the experiment are presented at the end of the notebook.  
Some code cells are included for future development and may not be directly relevant to this specific experiment.


# Scrap from the web portal of the dataset

You can use own-defined function to get the data from website, or use the pre-defined function in webutils.

Mind that you should check the `robots.txt` first to make sure if it is legal or allowed to scrap from this website.

In [1]:
from llm_metadata_harvester.webutils import extract_full_page_text
import nest_asyncio

url = "https://stac.ecodatacube.eu/veg_quercus.robur_anv.eml/collection.json?.language=en"

# Apply nest_asyncio to allow asyncio.run() in Jupyter
nest_asyncio.apply()

# Run the async function
full_text = await extract_full_page_text(url)

# Optionally display or save it
print(full_text[:10])  # Print the first 100 characters

EcoDataCub


Alternatively, you can download the HTML of the webpage and read it as a string.

In [6]:
file_path = "./webpages/Actual probability distribution for Quercus robur (2000–2020).html"

with open(file_path, "r") as file:
    full_text = file.read()

## 🔧 Environment Configuration

To configure your API keys or other environment variables, you can use a `.env` file or set them directly in your shell.

### 📄 Using a `.env` File

Place the `.env` file in **one** of the following locations:

- The **root directory** of your project  
- The **same directory** as the script you're running  
- Or any directory, **as long as it's the current working directory**

> ℹ️ The `load_dotenv()` function automatically looks for a `.env` file in the current working directory by default.

#### 💡 Example `.env` File
```env
OPENAI_API_KEY=your_api_key_here
ANOTHER_SECRET=value_here
```

### Using global environment variable

Alternatively, you can set environment variables directly in your shell:

```bash
export OPENAI_API_KEY=your_api_key_here
```

In [2]:
from tqdm import tqdm
from llm_metadata_harvester.harvester_operations import extract_entities
from llm_metadata_harvester.llm_client import LLMClient
from dotenv import load_dotenv

# can put your .env file in the root of the project
# or in the same directory as this script
# or set the environment variables directly in your shell
# load_dotenv() will look for a .env file in the current directory
load_dotenv()

True

## Metadata Fields

The metadata fields defined below follow the **Croissant data standard**.

In [3]:
# Define the metadata fields and their descriptions
# These fields are from croissant data standard
meta_field_dict = {
    "description": "Description of the dataset.",
    "license": "The license of the dataset. Croissant recommends using the URL of a known license, e.g., one of the licenses listed at https://spdx.org/licenses/.",
    "name": "The name of the dataset.",
    "creator": "The creator(s) of the dataset.",
    "datePublished": "The date the dataset was published.",
    "keywords": "A set of keywords associated with the dataset, either as free text, or a DefinedTerm with a formal definition.",
    "publisher": "The publisher of the dataset, which may be distinct from its creator.",
    "sameAs": "The URL of another Web resource that represents the same dataset as this one.",
    "dateModified": "The date the dataset was last modified.",
    "inLanguage": "The language(s) of the content of the dataset."
}

In [2]:
from llm_metadata_harvester.harvester_operations import metadata_harvest
from llm_metadata_harvester.standards import LTER_LIFE_STANDARD

extracted_metadata = await metadata_harvest(
    model_name="gemini-2.5-flash-preview-05-20",
    url = "https://stac.ecodatacube.eu/veg_quercus.robur_anv.eml/collection.json?.language=en",
    metadata_standard=LTER_LIFE_STANDARD)

extracted_metadata

{'Metadata date': 'N/A;',
 'Metadata language': 'English;',
 'Responsible organization metadata': 'Opengeohub;',
 'Landing page': 'https://doi.org/10.5281/zenodo.5887415;',
 'Title': 'Actual probability distribution for Quercus robur (2000–2020);',
 'Description': 'Actual Natural Vegetation (ANV): probability of occurrence for the Pedunculate oak in its realized environment for the period 2000 - 2033;',
 'Unique Identifier': 'https://doi.org/10.5281/zenodo.5887415;',
 'Resource type': 'Species distribution model;',
 'Keywords': 'species distribution model, tree species, landsat;',
 'Data creator': 'Carmelo Bonannella;',
 'Data contact point': 'carmelo.bonannella@opengeohub.org;',
 'Data publisher': 'Opengeohub;',
 'Spatial coverage': 'N/A;',
 'Spatial resolution': 'N/A;',
 'Spatial reference system': 'N/A;',
 'Temporal coverage': '2000-01-01 00:00:00 UTC – 2020-12-31 00:00:00 UTC;',
 'Temporal resolution': 'Multi-year intervals;',
 'License': 'CC-BY-SA-4.0;',
 'Access rights': 'Public 

## LLM Client Support

The `llm client` currently supports **OpenAI** and **Gemini** models.

To use other models, you can define your own LLM client class.  
Your custom class should implement a `chat` method that returns a string as the LLM response.

In [5]:
llm = LLMClient(model_name="gemini-2.5-flash-preview-05-20", temperature=0.0)

clean_nodes = extract_entities(
    text=full_text,
    meta_field_dict=meta_field_dict,
    llm=llm
)

The output of `extract_entities` looks like a list of lists of dictionaries, where each dictionary is structured like this:

```python
'license': [{'entity_name': 'license',
             'entity_value': 'CC-BY-SA-4.0',
             'source_id': 'chunk_0',
             'file_path': 'unknown_source'}]
```

In [1]:
from llm_metadata_harvester.checks import *
metadata = {"title": "Quercus robur"}
raw_text = "Actual probability distribution for Quercus robur (2000–2020)."
print("Exact match:", check_exist(metadata, raw_text))  # Should print True

# Example 2: Fuzzy match
metadata = {"title": "Quercus roburrr"}
raw_text = "Actual probability distribution for Quercus roburr (2000–2020)."
print("Fuzzy match:", check_exist(metadata, raw_text, threshold=0.8))  # Should print True

# Example 3: No match
metadata = {"title": "Pinus sylvestris"}
raw_text = "Actual probability distribution for Quercus robur (2000–2020)."
print("No match:", check_exist(metadata, raw_text))  # Should print False

# Example 4: Multiple metadata values
metadata = {"title": "Quercus robur", "author": "John Smith"}
raw_text = "John Smith studied the distribution of Quercus robur."
print("Multiple values:", check_exist(metadata, raw_text))  # Should print True

# Example 5: Empty metadata
metadata = {}
raw_text = "Some text."
print("Empty metadata:", check_exist(metadata, raw_text))  # Should print False

Exact match: {'title': True}
Fuzzy match: {'title': True}
No match: {'title': False}
Multiple values: {'title': True, 'author': True}


ValueError: extracted_metadata is empty

In [1]:
from llm_metadata_harvester.checks import *

extracted_metadata = {"title": "Quercus robur"}
metadata_definition = {"title": "Quercus robur"}
print("Exact repeat:", check_repeat_prompt(extracted_metadata, metadata_definition))  # (True,)

# Example 2: Fuzzy repeat
extracted_metadata = {"title": "Quercus roburr"}
metadata_definition = {"title": "Quercus robur"}
print("Fuzzy repeat:", check_repeat_prompt(extracted_metadata, metadata_definition, threshold=0.8))  # (True,)

# Example 3: No repeat
extracted_metadata = {"title": "Pinus sylvestris"}
metadata_definition = {"title": "Quercus robur"}
print("No repeat:", check_repeat_prompt(extracted_metadata, metadata_definition))  # (False,)

# Example 4: Multiple keys
extracted_metadata = {"title": "Quercus robur", "author": "John Smith"}
metadata_definition = {"title": "Quercus robur", "author": "Jane Doe"}
print("Multiple keys:", check_repeat_prompt(extracted_metadata, metadata_definition))  # (True, False)

# Example 5: Empty value in metadata
extracted_metadata = {"title": "", "author": None}
metadata_definition = {"title": "Quercus robur", "author": "Jane Doe"}
print("Empty value in metadata:", check_repeat_prompt(extracted_metadata, metadata_definition))  # (False, False)


Exact repeat: {'title': True}
Fuzzy repeat: {'title': True}
No repeat: {'title': False}
Multiple keys: {'title': True, 'author': False}
Empty value in metadata: {'title': False, 'author': False}


In [ ]:
from llm_metadata_harvester.harvester_operations import metadata_harvest
from dotenv import load_dotenv
from llm_metadata_harvester.standards import LTER_LIFE_STANDARD
import nest_asyncio
# Apply nest_asyncio to allow asyncio.run() in Jupyter
nest_asyncio.apply()

load_dotenv()
metadata = await metadata_harvest(
    model_name="gemini-2.5-flash-preview-05-20",
    url="https://stac.ecodatacube.eu/veg_quercus.robur_anv.eml/collection.json?.language=en",
    meta_field_dict=meta_field_dict
)
# default lter-life, 
# string -> 

In [ ]:
metadata

{'description': 'Actual Natural Vegetation (ANV) refers to the probability of occurrence for the Pedunculate oak in its realized environment for the period 2000-2033.;',
 'license': 'CC-BY-SA-4.0;',
 'name': 'Actual Probability Distribution for Quercus Robur;',
 'creator': 'Carmelo Bonannella;',
 'datePublished': '2000-01-01;',
 'keywords': 'Species Distribution Model, Tree Species, Landsat;',
 'publisher': 'OpenGeoHub;',
 'sameAs': 'https://doi.org/10.5281/zenodo.5887415;',
 'dateModified': '2020-12-31;',
 'inLanguage': 'English;'}